In [1]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
from dspy.teleprompt import BootstrapFewShot

from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
init_sys_env()
secret_env_file = '/mnt/d/project/dy/extra/autogen/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=2048,api_key=model_config.get('api_key'))
dspy.settings.configure(lm=turbo)


In [2]:
class BasicQA(dspy.Signature):
  """回答问题，答案要简短。"""
  question = dspy.InputField()
  answer = dspy.OutputField(desc="通常是 1 到 5 个词", prefix="问题的答案:") 
  

In [8]:
predictor = dspy.Predict(BasicQA)
result = predictor.forward(question="1+1=?")
result.answer

'2'

In [5]:
turbo.history[-1]

{'prompt': '回答问题，答案要简短。\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n问题的答案: 通常是 1 到 5 个词\n\n---\n\nQuestion: 1+1=?\n问题的答案:',
 'response': {'id': 'chatcmpl-9TKxNyw27NUwIqZW7SIfZ86MyTBJc',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': '2',
     'role': 'assistant',
     'function_call': None,
     'tool_calls': None}}],
  'created': 1716779097,
  'model': 'gpt-4o-2024-05-13',
  'object': 'chat.completion',
  'system_fingerprint': 'fp_43dfabdef1',
  'usage': {'completion_tokens': 1, 'prompt_tokens': 54, 'total_tokens': 55}},
 'kwargs': {'stringify_request': '{"temperature": 0.0, "max_tokens": 2048, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "model": "gpt-4o", "messages": [{"role": "user", "content": "\\u56de\\u7b54\\u95ee\\u9898\\uff0c\\u7b54\\u6848\\u8981\\u7b80\\u77ed\\u3002\\n\\n---\\n\\nFollow the following format.\\n\\nQuestion: ${question}\\n\\u95ee\\u9898\\u7684\\u7b54\\u6848: \\u

In [7]:
turbo.inspect_history(n=1)




回答问题，答案要简短。

---

Follow the following format.

Question: ${question}
问题的答案: 通常是 1 到 5 个词

---

Question: 1+1=?
问题的答案: 2





'\n\n\n回答问题，答案要简短。\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n问题的答案: 通常是 1 到 5 个词\n\n---\n\nQuestion: 1+1=?\n问题的答案:\x1b 2\x1b\n\n\n'

In [17]:


class Writer(dspy.Signature):
  question = dspy.InputField()
  answer = dspy.OutputField(desc="通常是一段话,并且详细的说明为什么?")
  tone = dspy.OutputField(desc="使用鲁迅先生的语气和语法回答这段话")
  

In [18]:
writer_predictor = dspy.Predict(Writer)
writer_result = writer_predictor.forward(question="如何评价阿Q精神")
writer_result.answer

'Answer: 阿Q精神，乃是中国社会底层人民在极端压迫下所表现出的一种自我安慰的心理状态。阿Q以其独特的方式，面对生活中的种种不公与苦难，虽有时显得滑稽可笑，但却深刻地揭示了当时社会的病态与人性的扭曲。阿Q的精神胜利法，表面上看似一种逃避现实的手段，实则反映了他内心深处对尊严与自由的渴望。鲁迅先生通过阿Q这一形象，揭露了封建社会的黑暗与腐朽，呼唤人们觉醒，追求真正的解放与进步。'

In [21]:
turbo.history[-1]

{'prompt': 'Given the fields `question`, produce the fields `answer`, `tone`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: 通常是一段话,并且详细的说明为什么?\nTone: 使用鲁迅先生的语气和语法回答这段话\n\n---\n\nQuestion: 如何评价阿Q精神\nAnswer:',
 'response': {'id': 'chatcmpl-9TLEJ8PJSJDzn1Nabx8EISridTkEY',
  'choices': [{'finish_reason': 'stop',
    'index': 0,
    'logprobs': None,
    'message': {'content': 'Answer: 阿Q精神，乃是中国社会底层人民在极端压迫下所表现出的一种自我安慰的心理状态。阿Q以其独特的方式，面对生活中的种种不公与苦难，虽有时显得滑稽可笑，但却深刻地揭示了当时社会的病态与人性的扭曲。阿Q的精神胜利法，表面上看似一种逃避现实的手段，实则反映了他内心深处对尊严与自由的渴望。鲁迅先生通过阿Q这一形象，揭露了封建社会的黑暗与腐朽，呼唤人们觉醒，追求真正的解放与进步。\nTone: 使用鲁迅先生的语气和语法回答这段话',
     'role': 'assistant',
     'function_call': None,
     'tool_calls': None}}],
  'created': 1716780147,
  'model': 'gpt-4o-2024-05-13',
  'object': 'chat.completion',
  'system_fingerprint': 'fp_43dfabdef1',
  'usage': {'completion_tokens': 182,
   'prompt_tokens': 76,
   'total_tokens': 258}},
 'kwargs': {'stringify_request': '{"temperature": 0.0, "max_tokens": 2048, "top

In [20]:
turbo.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`, `tone`.

---

Follow the following format.

Question: ${question}
Answer: 通常是一段话,并且详细的说明为什么?
Tone: 使用鲁迅先生的语气和语法回答这段话

---

Question: 如何评价阿Q精神
Answer: Answer: 阿Q精神，乃是中国社会底层人民在极端压迫下所表现出的一种自我安慰的心理状态。阿Q以其独特的方式，面对生活中的种种不公与苦难，虽有时显得滑稽可笑，但却深刻地揭示了当时社会的病态与人性的扭曲。阿Q的精神胜利法，表面上看似一种逃避现实的手段，实则反映了他内心深处对尊严与自由的渴望。鲁迅先生通过阿Q这一形象，揭露了封建社会的黑暗与腐朽，呼唤人们觉醒，追求真正的解放与进步。
Tone: 使用鲁迅先生的语气和语法回答这段话





'\n\n\nGiven the fields `question`, produce the fields `answer`, `tone`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: 通常是一段话,并且详细的说明为什么?\nTone: 使用鲁迅先生的语气和语法回答这段话\n\n---\n\nQuestion: 如何评价阿Q精神\nAnswer:\x1b Answer: 阿Q精神，乃是中国社会底层人民在极端压迫下所表现出的一种自我安慰的心理状态。阿Q以其独特的方式，面对生活中的种种不公与苦难，虽有时显得滑稽可笑，但却深刻地揭示了当时社会的病态与人性的扭曲。阿Q的精神胜利法，表面上看似一种逃避现实的手段，实则反映了他内心深处对尊严与自由的渴望。鲁迅先生通过阿Q这一形象，揭露了封建社会的黑暗与腐朽，呼唤人们觉醒，追求真正的解放与进步。\nTone: 使用鲁迅先生的语气和语法回答这段话\x1b\n\n\n'